# Booking.com WSDM WebTour 2021 Challenge demo

In [1]:
import pandas as pd

## Load train set

In [15]:
train_set = pd.read_csv('booking_train_set.csv', index_col=[0]).sort_values(by=['utrip_id','checkin'])
print(train_set.shape[0])

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


1166835


* ALT - filter for only train cities with multiple appearances, (after first step?/row_num)
* Doesn't affect top k result 

In [17]:
df["checkin"] = pd.to_datetime(df["checkin"],infer_datetime_format=True)
df["row_num"] = df.groupby("utrip_id")["checkin"].rank(ascending=True,pct=False).astype(int)
utrip_counts = df["utrip_id"].value_counts()
df["total_rows"] = df["utrip_id"].map(utrip_counts)
df

,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,c,row_num,total_rows
542547,1000027,2016-08-14,2016-08-16,15626,desktop,7168,Elbonia,Gondal,1000027_1,1113,1,3
542548,1000027,2016-08-16,2016-08-18,60902,desktop,7168,Elbonia,Gondal,1000027_1,62,2,3
542549,1000027,2016-08-18,2016-08-21,30628,desktop,253,Elbonia,Gondal,1000027_1,72,3,3
1061281,1000033,2016-04-09,2016-04-11,38677,mobile,359,Gondal,Cobra Island,1000033_1,2247,1,5
1061282,1000033,2016-04-11,2016-04-12,52089,desktop,384,Gondal,Cobra Island,1000033_1,150,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1120386,999855,2016-04-22,2016-04-24,33408,mobile,359,Gondal,Fook Island,999855_1,779,8,11
1120390,999855,2016-04-27,2016-04-29,63729,mobile,359,Gondal,Fook Island,999855_1,76,9,11
1120391,999855,2016-04-29,2016-05-01,44489,mobile,359,Gondal,Fook Island,999855_1,479,10,11
1120392,999855,2016-05-01,2016-05-02,20345,mobile,359,Gondal,Fook Island,999855_1,2391,11,11


In [3]:
freq = train_set["city_id"].value_counts()
train_set["c"] = train_set["city_id"].map(freq)
print(train_set["c"].describe())
train_set = train_set.loc[train_set["c"]>50]
print(train_set.shape[0])

count    1.166835e+06
mean     1.666303e+03
std      2.507403e+03
min      1.000000e+00
25%      8.700000e+01
50%      5.430000e+02
75%      1.970000e+03
max      1.124200e+04
Name: c, dtype: float64
940108


In [4]:
train_set.head()

,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,c
542547,1000027,2016-08-14,2016-08-16,15626,desktop,7168,Elbonia,Gondal,1000027_1,1113
542548,1000027,2016-08-16,2016-08-18,60902,desktop,7168,Elbonia,Gondal,1000027_1,62
542549,1000027,2016-08-18,2016-08-21,30628,desktop,253,Elbonia,Gondal,1000027_1,72
1061281,1000033,2016-04-09,2016-04-11,38677,mobile,359,Gondal,Cobra Island,1000033_1,2247
1061282,1000033,2016-04-11,2016-04-12,52089,desktop,384,Gondal,Cobra Island,1000033_1,150


### Load testset

In [6]:
df = train_set


In [7]:
df["row_num"] = df.groupby("utrip_id")["checkin"].rank(ascending=True,pct=False)
df["total_rows"] = df.groupby("utrip_id").transform("size")
df.tail()

DataError: No numeric types to aggregate

In [37]:
test_set = pd.read_csv('sample_test_set.csv').sort_values(by=['utrip_id','checkin'])

In [38]:
test_set.head()

,user_id,checkin,checkout,device_class,affiliate_id,booker_country,utrip_id,row_num,total_rows,city_id,hotel_country
0,1000066,21/07/2016,23/07/2016,desktop,9924,Gondal,1000066_5,1,4,56430,Urkesh
1,1000066,23/07/2016,25/07/2016,desktop,9924,Gondal,1000066_5,2,4,11543,Urkesh
2,1000066,25/07/2016,28/07/2016,desktop,9924,Gondal,1000066_5,3,4,5797,Urkesh
3,1000066,28/07/2016,31/07/2016,mobile,2436,Gondal,1000066_5,4,4,0,NaN
4,1000270,08/02/2016,09/02/2016,mobile,9452,The Devilfire Empire,1000270_5,1,4,50075,The Devilfire Empire


### Generate Dummy Predictions - use top 4 cities in the trainset as benchmark recommendation

In [17]:
st = ["user_id","utrip_steps_from_end","user_id"]

['utrip_steps_from_end']

In [25]:
df = pd.read_csv('sub.csv')
df["r_num"] = df.groupby("utrip_id")["city_id"].rank(ascending=True,pct=False)
df = df.pivot(index='utrip_id', columns='r_num', values='city_id').reset_index() ## , values='score'
df.columns = ["utrip_id",'city_id_1','city_id_2','city_id_3','city_id_4']
df

,utrip_id,city_id_1,city_id_2,city_id_3,city_id_4
0,1000066_2,2748,22065,41971,46854
1,1000270_1,24507,32392,50075,51517
2,1000441_1,46411,47759,50457,52933
3,100048_1,13530,21480,26235,36805
4,1000543_1,16521,29770,44869,52818
...,...,...,...,...,...
70657,999674_1,657,10994,51765,53207
70658,999797_1,4476,20392,23542,51291
70659,999862_1,2748,22065,41971,46854
70660,999911_1,23921,29319,48483,65322


In [ ]:
pd.DataFrame([topcities]*test_trips.shape[0]
                                 , columns= ['city_id_1','city_id_2','city_id_3','city_id_4'])

In [39]:
topcities = train_set.city_id.value_counts().index[:4]
topcities

Int64Index([47499, 23921, 36063, 17013], dtype='int64')

In [40]:
test_trips = (test_set[['utrip_id']].drop_duplicates()).reset_index().drop('index', axis=1)

In [41]:
cities_prediction = pd.DataFrame([topcities]*test_trips.shape[0]
                                 , columns= ['city_id_1','city_id_2','city_id_3','city_id_4'])

### Create Submission file according to the format

In [42]:
submission = pd.concat([test_trips,cities_prediction], axis =1)
submission

,utrip_id,city_id_1,city_id_2,city_id_3,city_id_4
0,1000066_5,47499,23921,36063,17013
1,1000270_5,47499,23921,36063,17013
2,1000441_5,47499,23921,36063,17013
3,100048_5,47499,23921,36063,17013


In [43]:
submission.to_csv('submission.csv',index=False)

## Read submission file and ground truth

In [44]:
ground_truth = pd.read_csv('sample_truth.csv', index_col =[0])
submission = pd.read_csv('submission.csv', index_col =[0])

In [45]:
ground_truth

,city_id,hotel_country
utrip_id,,
1000066_5,41971,Urkesh
1000270_5,23921,Cobra Island
1000441_5,50457,Osterlich
100048_5,17013,Borginia


## Evaluate - use accuracy at 4 to evaluate the prediction

In [46]:
def evaluate_accuracy_at_4(submission,ground_truth):
    '''checks if the true city is within the four recommended cities'''
    data_to_eval = submission.join(ground_truth,on='utrip_id')
    hits = data_to_eval.apply(
        lambda row: row['city_id'] in (row[['city_id_1', 'city_id_2', 'city_id_3', 'city_id_4']].values),
            axis = 1)
    return hits.mean()

In [47]:
evaluate_accuracy_at_4(submission,ground_truth) ### .5 for default 
### .5 if filtering out cities appearing less than 50 times - irrelevant, since those don't appear anyway

0.5